<a href="https://colab.research.google.com/github/leul12/Data_Exploration/blob/main/tweet_stalking_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import itertools
from datetime import datetime
import datetime
import time
from pandas.io.json import json_normalize
import json
# from tqdm.notebook import tqdm_notebook
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

## Let's Check the version 🐍

In [2]:
from platform import python_version
print(python_version())

3.10.4


## Okay now we can do our **job**

1.   Let's install the 🔨
2.   Extract the 🗄️
3.   And So on



# ⬆️ install the tool 🔨

In [3]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-r64yebs6
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-r64yebs6
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit b515a66b937276eafff41cf6b897c53ecf3d54a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


## 🤔   **how to extract the tweets ?** no worries ⬇️

In [4]:
import snscrape.modules.twitter as sntwitter

# okay now some magic to get the actual 🐦 using the **itertools** 😉

In [5]:
df = pd.DataFrame(itertools.islice(
    sntwitter.TwitterSearchScraper('#vaccine').get_items(), 300))
# let's Test what kind of data we can get from it (if it works)
df.head(2)

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,https://twitter.com/AIBiotechnology/status/161...,2023-01-24 08:00:00+00:00,@John_Anti_Trump No vaccine here. Out partying...,@John_Anti_Trump No vaccine here. Out partying...,1617794332446330882,"{'username': 'AIBiotechnology', 'id': 10018530...",0,0,0,0,...,1.617212e+18,"{'username': 'John_Anti_Trump', 'id': 13296114...","[{'username': 'John_Anti_Trump', 'id': 1329611...",None,None,"[covid, vaccine, vaccineinjuries]",None,None,NaN,None
1,https://twitter.com/Geostrategic777/status/161...,2023-01-24 07:58:10+00:00,UNDERCOVER VIDEO: Pfizer Scientists Knew That ...,UNDERCOVER VIDEO: Pfizer Scientists Knew That ...,1617793870112358402,"{'username': 'Geostrategic777', 'id': 83861925...",0,0,0,0,...,NaN,None,None,None,None,"[Covid, Vaccine, CovidVaccine]",None,{'title': 'UNDERCOVER VIDEO: Pfizer Scientists...,2.0,None


# okay Now that we got this out of the way what now 🤔


*   we check data heads
*   clean some columns [user , hastags , date , mentionedUsers & inReplyToUser ]
*   and take it form there 
*   🚶‍♂️



In [6]:
dfuser= pd.json_normalize(df['user'])
dfuser.head(2)

,username,id,displayname,rawDescription,renderedDescription,descriptionLinks,verified,created,followersCount,friendsCount,...,location,protected,profileImageUrl,profileBannerUrl,label,link.text,link.url,link.tcourl,link.indices,link
0,AIBiotechnology,100185304,AI Biotechnology correlation suppose not causa...,All posts and retweets are NOT endorsements. I...,All posts and retweets are NOT endorsements. I...,None,False,2009-12-29 09:36:31+00:00,5168,5517,...,"Raleigh, NC",False,https://pbs.twimg.com/profile_images/154180796...,https://pbs.twimg.com/profile_banners/10018530...,None,bitira.go2cloud.org/aff_c?offer_id…,https://bitira.go2cloud.org/aff_c?offer_id=5&a...,https://t.co/4Z5aglQLRJ,"(0, 23)",NaN
1,Geostrategic777,838619254530408449,Mark Espinola 🇺🇸,"Commodity trader, #SlavaUkraini, Trump, Israel...","Commodity trader, #SlavaUkraini, Trump, Israel...",None,False,2017-03-06 05:16:25+00:00,846,1856,...,NYC,False,https://pbs.twimg.com/profile_images/876755956...,https://pbs.twimg.com/profile_banners/83861925...,None,NaN,NaN,NaN,NaN,NaN


# NOw Let's Take what we need and add the to the main Data Frame ⚙️

In [7]:
dfuser.drop(['id','displayname','rawDescription','renderedDescription','descriptionLinks','location','statusesCount','favouritesCount','listedCount',
                  'mediaCount','link','profileImageUrl','profileBannerUrl','label','link.text','link.url','link.tcourl','link.indices'], axis = 1, inplace = True)
dfuser.head(2)

,username,verified,created,followersCount,friendsCount,protected
0,AIBiotechnology,False,2009-12-29 09:36:31+00:00,5168,5517,False
1,Geostrategic777,False,2017-03-06 05:16:25+00:00,846,1856,False


# now that we 🫧 the 🧫  time to add it to the clean data sets

In [8]:
df2=df.drop(['user','renderedContent','sourceUrl','hashtags','cashtags','card','inReplyToTweetId','inReplyToUser','mentionedUsers','coordinates','links','media','retweetedTweet','quotedTweet','place'],axis=1)
df2.head(2)

,url,date,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,viewCount,vibe
0,https://twitter.com/AIBiotechnology/status/161...,2023-01-24 08:00:00+00:00,@John_Anti_Trump No vaccine here. Out partying...,1617794332446330882,0,0,0,0,1617212144432422912,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,NaN,None
1,https://twitter.com/Geostrategic777/status/161...,2023-01-24 07:58:10+00:00,UNDERCOVER VIDEO: Pfizer Scientists Knew That ...,1617793870112358402,0,0,0,0,1617793870112358402,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,2.0,None


In [9]:
result = pd.concat([df2, dfuser], axis=1)
result.head(2)

,url,date,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,viewCount,vibe,username,verified,created,followersCount,friendsCount,protected
0,https://twitter.com/AIBiotechnology/status/161...,2023-01-24 08:00:00+00:00,@John_Anti_Trump No vaccine here. Out partying...,1617794332446330882,0,0,0,0,1617212144432422912,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,NaN,None,AIBiotechnology,False,2009-12-29 09:36:31+00:00,5168,5517,False
1,https://twitter.com/Geostrategic777/status/161...,2023-01-24 07:58:10+00:00,UNDERCOVER VIDEO: Pfizer Scientists Knew That ...,1617793870112358402,0,0,0,0,1617793870112358402,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,2.0,None,Geostrategic777,False,2017-03-06 05:16:25+00:00,846,1856,False


# ⏱️ manuplation

In [10]:
result.set_index('date').head(2)

,url,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,viewCount,vibe,username,verified,created,followersCount,friendsCount,protected
date,,,,,,,,,,,,,,,,,,,
2023-01-24 08:00:00+00:00,https://twitter.com/AIBiotechnology/status/161...,@John_Anti_Trump No vaccine here. Out partying...,1617794332446330882,0,0,0,0,1617212144432422912,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,NaN,None,AIBiotechnology,False,2009-12-29 09:36:31+00:00,5168,5517,False
2023-01-24 07:58:10+00:00,https://twitter.com/Geostrategic777/status/161...,UNDERCOVER VIDEO: Pfizer Scientists Knew That ...,1617793870112358402,0,0,0,0,1617793870112358402,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,2.0,None,Geostrategic777,False,2017-03-06 05:16:25+00:00,846,1856,False
